<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/stable_baselines_ppo_trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/DLR-RM/stable-baselines3@feat/gymnasium-support
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib@feat/gymnasium-support
!pip install yfinance
!pip install gymnasium


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/DLR-RM/stable-baselines3 (to revision feat/gymnasium-support) to /tmp/pip-req-build-sr92np3p
  Running command git clone --filter=blob:none --quiet https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-sr92np3p
  Running command git checkout -q feat/gymnasium-support
  error: pathspec 'feat/gymnasium-support' did not match any file(s) known to git
  error: subprocess-exited-with-error
  
  × git checkout -q feat/gymnasium-support did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git checkout -q feat/gymnasium-support did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Looking in indexes: https://pypi.org/

In [2]:
!pip install sb3-contrib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import os

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')
data_path = '/content/drive/My Drive/ml-trde-notebooks/data'
model_path = "/content/drive/My Drive/ml-trde-notebooks/models"



Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import math

from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import norm

import multiprocessing

from stable_baselines3 import PPO
from sb3_contrib import RecurrentPPO
from stable_baselines3.ppo.policies import MlpPolicy

from stable_baselines3.common.evaluation import evaluate_policy

import torch
import torch.nn as nn
import torch.optim as optim


cores = multiprocessing.cpu_count()
print("I has cores: ", cores)





I has cores:  12


In [5]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/backtest-common.ipynb'

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Modified
1679922395
1679922395.0
1679922395.0


In [6]:
logging_level = 1
output_file = open(os.path.join(data_path, 'ppo-env.txt'), 'w')
def to_file(*args):
  output_str = ' '.join(str(arg) for arg in args)
  output_file.write(output_str)
  output_file.flush()
  
def error(*args):
    if logging_level >= 0:
        to_file(*args)
        print(*args)

def info(*args):
    if logging_level >=1:
        to_file(*args)
        print(*args)

def verbose(*args):
    if logging_level >=2:
        to_file(*args)
        print(*args)

def debug(*args):
    if logging_level >=3:
        to_file(*args)
        print(*args)

In [7]:
def download_stocks(total):
  # We don't want to train against crypto
  return get_all_stock_timerseries_for_csv("training_tickers3.csv", 3500, total)

LENGTH_OF_STOCK_TRAINGING_DATA = 145 #I might need to fix this, but the model is tied to the number of symbols we trained on
def download_crypto():
  return get_all_product_timeseries(-1, 180, LENGTH_OF_STOCK_TRAINGING_DATA)

In [8]:
def get_data_for_training(num):

  from_disk = False
  use_crypto = False

  if from_disk:
    product_data = load_dict(use_crypto)
    first_key = list(product_data.keys())[0]
    length = len(product_data[first_key])
  elif use_crypto:
    product_data = download_crypto()
    save_dict(product_data, True)
  else:
    product_data = download_stocks(num)
    save_dict(product_data)

  min_len = min(len(df) for df in product_data.values())
  print("symbols:", len(product_data.keys()))
  print("min length:", min_len)
  for name, df in product_data.items():
      product_data[name] = df.head(min_len)

  product_data = dict(sorted(product_data.items(), reverse=True))

  return product_data

In [9]:
import numpy as np
import gymnasium as gym
from gymnasium import logger, spaces


class TraderEnv(gym.Env):
    
    def __init__(self, product, df, curriculum_code=1):
        super(TraderEnv, self).__init__()

        # Define the bounds for each column
        close_min, close_max = 0, np.inf  
        volume_min, volume_max = 0, np.inf  
        trend_min, trend_max = 0, np.inf  
        percentile_min, percentile_max = 0, 1

        # Define the observation space
        self.observation_space = spaces.Box(low=np.array([close_min, volume_min, trend_min, percentile_min]),
                                      high=np.array([close_max, volume_max, trend_max, percentile_max]),
                                      dtype=np.float32)

        # We have 3 actions, hold (0), long (1), short (2)
        self.action_space = spaces.Discrete(3)
        
        # Initialize environment state
        df = self.expand(df.copy())

        self.orig_timeseries = df
        self.timeseries = self.scale(df[["Date", "Close", "weighted-volume", "trend", "prob_above_trend"]])
        
        self._reset_vars()
        self.calculate_benchmark_metrics()


        # Don't add this here. Process the tickers into data files - you already have some.
        #self.lstm_internal = keras.models.load_model(model_path + "/lstm_cv-4.h15")
        #predictions = self.predict_trade(self.lstm_internal, self.timeseries[["Close", "weighted-volume"]])
        #self.timeseries["predicted-price"] = predictions
        
        self.product = product
        self.final = len(df)

        self.curriculum_code = curriculum_code
        self.rolling_score = 0

    
    def expert_opinion(self):
      df = self.timeseries

      def assign_action(prob):
          if prob >= 0.8:
              return 1
          elif prob <= 0.2:
              return 2
          else:
              return 0

      state_action_data = []
      for index, row in df.iterrows():
          state = row.values  # Keep all columns, including 'prob_above_trend', as part of the state
          action = assign_action(row["prob_above_trend"])
          state_action_data.append((state, action))

      return state_action_data

    def expert_opinion_df(self):
      df = self.timeseries
      
      # iterate over the time series
      def assign_action(prob):
          if prob >= 0.8:
              return 1
          elif prob <= 0.2:
              return 2
          else:
              return 0

      # Create the new column 'action' based on the values in 'prob_above_trend'
      df['action'] = df['prob_above_trend'].apply(assign_action)

    def calculate_benchmark_metrics(self):
      df = self.orig_timeseries
      row = df.iloc[0,:]
      price = self.get_price_with_slippage(row["Close"])
      self.benchmark_position_shares = math.floor(self.cash/price)
     
    def predict_trade(self, model, X):
      predicted = model.predict(X, verbose=0).flatten()
      return predicted

    def expand(self, df):
      # Perform seasonal decomposition
      result = seasonal_decompose(df['Close'], model='additive', period=90, extrapolate_trend='freq')

      # Add trend back to original time series
      df["trend"] = result.trend

      # Compute the residuals by subtracting the trend from the original time series
      residuals = result.resid

      # Fit a Gaussian distribution to the residuals
      mu, std = norm.fit(residuals)

      # Compute the probability of a value being above or below the trend line
      # for each point in the time series
      z_scores = residuals / std
      df["prob_above_trend"] = 1 - norm.cdf(z_scores)
      df["weighted-volume"] = df["Close"] * df["Volume"]
      return df

    def scale(self, timeseries):
      df = timeseries.reset_index()  # Reset the index of the DataFrame
      dates = df['Date']
      data_to_scale = df.drop(['Date', 'index'], axis=1)
      self.scaler = MinMaxScaler()
      scaled_data = self.scaler.fit_transform(data_to_scale)
      scaled_df = pd.concat([dates, pd.DataFrame(scaled_data, columns=data_to_scale.columns)], axis=1)
      return scaled_df.set_index('Date')  # Set the index back to 'Date'

    def _reset_vars(self):

        self._episode_ended = False        
        self.ledger = self.make_ledger_row()
        self.slippage = .01
        self.fee = .0025
        self.current_index = 0
        self.cash = 5000
        self.position_shares = 0
        self.cash_from_short = 0
        self.position_value = 0
        self.shares_owed = 0
        self.in_position = False
        self.in_long = False
        self.in_short = False
        self.last_action = -1
        self.long_profit = []
        self.short_profit = []
        self.long_entry = -1
        self.short_entry = -1
        self.rolling_score = 0

        

    def reset(self):
        # Reset the environment and return the initial time step
        self._reset_vars()
        return self._get_next_state(), {}
        #return self._get_next_state()

    def reset_test(self):
        self._reset_vars()
        return self._get_next_state()

    def step(self, action):
        
        action = int(action)
        info("_step:", self.current_index, " action: ", action)

        self.last_action = action
        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start a new episode.
            return self.reset()

        # Apply the action and update the environment state
        self._apply_action(action)
        

        if self._is_episode_ended():
            reward = self._get_reward2()
            info("final reward:", reward)
            return self._get_next_state(), reward, False, True, {}
            #return self._get_next_state(), reward, True, {}
        else:
            reward = self._get_reward2()
            info("current reward:", reward)
            return self._get_next_state(), reward, False, False, {}
            #return self._get_next_state(), reward, False, {}

    def _get_initial_state(self):
        # Return the initial state of the environment
        self.current_index = 0
        return self.env_block()

    def should_stop(self):
      # if cash is negative
      if (self.total_value() <= 0):
        error("Bankrupt.")
        return True
      return False


    def _apply_action(self, action):
        # Advance the environment by one time step and return the observation, reward, and done flag
        self.current_index += 1
        verbose("step:", "index:", self.current_index, " of: ", self.final-1, "action: ", int(action))
        self.update_position_value()
        if (self.current_index >= self.final - 1): #or  self.should_stop()
          error("********MARKING DONE", "index:", self.current_index, " of: ", self.final-1, " cash: ", self.cash, " value: ", self.position_value)
          if (self.position_shares != 0):
            verbose("done so closing position")
            self.close_position()
          self._episode_ended = True
        else:
          self._episode_ended = False

        # AI says hold
        if (action == 0):
          info("holding action = 0.")
          self.closed_position = False
        # AI says long but we're already in a position
        elif (action == 1 and self.in_long):
          info("holding long.")
          self.closed_position = False
        # AI says long, we're not in a position, so buy
        elif (action == 1 and not self.in_position):
          info("opening long.")
          self.open_position()
          self.closed_position = False
        #AI says long, but we're short. Close the short, open a long.
        elif (action == 1 and self.in_short):
          info("closing short to open long.")
          self.closed_position = True
          self.close_short() 
          self.open_position()
        #AI says short, but we're already short  
        elif (action == 2 and self.in_short):
          info("holding short.")
          self.closed_position = False
        #AI says short and we're not in a position so exit 
        elif (action == 2 and not self.in_position):
          info("opening short.")
          self.open_short()
          self.closed_position = False
        #AI says short but we're long, close it
        elif (action == 2 and self.in_long):
          info("closing long to open short")
          self.close_position()
          info("opening short.")
          self.open_short()
          self.closed_position = True
        else: #assume hold
          error("unknown state! holding:", action, self.in_position, self.in_long, self.in_short)      
          self.closed_position = False
        self.update_position_value()
    
    def _get_next_state(self):
        # Calculate and return the next state based on the current state and action taken
        return self.env_block()

    def get_price_with_slippage(self, price):
        return price + (price * self.slippage)


    def make_ledger_row(self):
        ledger = pd.DataFrame()
        ledger["Date"] = []
        ledger["Side"] = []
        ledger["Action"]  = []
        ledger["Price"] = []
        ledger["Profit"] = []
        ledger["Fee"] = []
        ledger["Value"] = []
        
        return ledger

    #todo you need to subtract fee from cash
    def open_position(self):
      self.in_position = True
      self.in_long = True
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      self.position_shares = math.floor(self.cash/price)
      fee = self.cash * self.fee
      self.cash = 0    
      self.long_entry = price  
      ledger_row = self.make_ledger_row()
      ledger_row["Date"] = [row["Date"]]
      ledger_row["Side"] = ["long"]
      ledger_row["Action"] = ["enter"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [fee]
      ledger_row["Profit"] = [0]
      ledger_row["Value"] = [self.total_value()]
      
      self.ledger = pd.concat([self.ledger, ledger_row])

    def open_short(self):
      self.in_position = True
      self.in_short = True
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      max_short_pos = math.floor(self.cash/price)
      self.shares_owed = max_short_pos
      self.cash_from_short = (self.shares_owed * price)   
      fee = self.cash * self.fee 
      self.cash = self.cash + self.cash_from_short
      self.short_entry = price
      verbose("Added cash on short: ", self.shares_owed * price, " total: ", self.cash, " took share debt:", self.shares_owed)  
      ledger_row = self.make_ledger_row()
      ledger_row["Date"] = [row["Date"]]
      ledger_row["Side"] = ["short"]
      ledger_row["Action"] = ["enter"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [fee]
      ledger_row["Profit"] = [0]
      ledger_row["Value"] = [self.total_value()]
      self.ledger = pd.concat([self.ledger, ledger_row])

    def close_position(self):
      self.in_position = False
      self.in_long = False
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      value = price * self.position_shares
      fee = value * self.fee 
      self.position_shares = 0
      self.cash = self.cash + value
      self.last_profit = (price - self.long_entry)/self.long_entry
      self.long_profit.append(self.last_profit)
      ledger_row = self.make_ledger_row()
      ledger_row["Date"] = [row["Date"]]
      ledger_row["Side"] = ["long"]
      ledger_row["Action"] = ["exit"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [fee]
      ledger_row["Profit"] = [self.last_profit]
      ledger_row["Value"] = [self.total_value()]
      self.ledger = pd.concat([self.ledger, ledger_row])

    def close_short(self):
      self.in_position = False
      self.in_short = False
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      price = self.get_price_with_slippage(row["Close"])
      value = price * self.shares_owed
      fee = value * self.fee
      self.shares_owed = 0
      self.cash_from_short = 0
      self.cash = self.cash - value
      self.last_profit = (self.short_entry - price)/self.short_entry
      self.short_profit.append(self.last_profit)
      ledger_row = self.make_ledger_row()
      ledger_row["Date"] = [row["Date"]]
      ledger_row["Side"] = ["short"]
      ledger_row["Action"] = ["exit"]
      ledger_row["Price"] = [price]
      ledger_row["Fee"] = [fee]
      ledger_row["Profit"] = [self.short_entry - price]
      ledger_row["Value"] = [self.total_value()]
      self.ledger = pd.concat([self.ledger, ledger_row])


    def _is_episode_ended(self):
        return self._episode_ended
        
    def update_position_value(self):
      df = self.orig_timeseries
      row = df.iloc[self.current_index,:]
      self.position_value = (row["Close"] * self.position_shares) - (row["Close"] * self.shares_owed)
      self.benchmark_value = row["Close"] * self.benchmark_position_shares

    def _get_reward2(self):
      '''
        What components should go into the reward?
        We can encode some of our desired behavior here, for example if we are at 99% chance of above trend we could reward long and vice versa
        We can remove cash from the reward calculation and simply treat it well we initially trade from. Any profits go into cash from trades, initial cash is penalized
        Consistency - making lots of profitable trades is better then big bets or not betting
      '''
      score = 0
      
      component1 = self.is_probable_set_up() 
      self.rolling_score+= component1


      '''
      consistency_bonus = self.calculate_consistency_bonus()
      score+=consistency_bonus
      info("apply consistency_bonus: ", consistency_bonus)
      
      if self.curriculum_code == 2:
        return score
      
      close_bonus = self.calculate_close_bonus()
      score+=close_bonus
      info("apply close_bonus: ", close_bonus)
      
      if self.curriculum_code == 3:
        return score
        
      current_portfolio_value = self.total_value()
      percentage_change = ((current_portfolio_value - self.benchmark_value) / self.benchmark_value) * 100
      score+=percentage_change
      info("apply portfolio_value: ", percentage_change)
      '''
      return self.rolling_score

    def calculate_consistency_bonus(self):
      profitable_longs = list(filter(lambda x: x >= 0, self.long_profit))
      profitable_shorts = list(filter(lambda x: x >= 0, self.short_profit))
      profitable_trades = len(profitable_longs) + len(profitable_shorts)
      max = len(self.long_profit) + len(self.short_profit)
      if max == 0:
        return 0
      return profitable_trades/max * 100

    def calculate_close_bonus(self):
      if self.closed_position and self.last_profit>0:
        return self.last_profit * 10
      else:
        return 0

    def is_probable_set_up(self):
        df = self.orig_timeseries
        row = df.iloc[self.current_index, :]
        prob = row["prob_above_trend"]
        action = self.last_action

        print("action:", action, " prob:", prob)
        if action == 1 and prob >= 0.8:
            return 1 # reward a highly probable long
        elif action == 2 and prob <= 0.1:
            return 1 # reward a highly probable short
        elif action == 0 and (0.1 > prob or prob < 0.8):
            return 0.5 # reward holding when probability is moderate
        else:
            return -1 # penalize for taking actions against the criteria


    def _get_reward(self):
        df = self.orig_timeseries
        row = df.iloc[self.current_index,:]
        current_portfolio_value = self.total_value()
       
       
        #compare to a bench mark
        percentage_change = ((current_portfolio_value - self.benchmark_value) / self.benchmark_value) * 100
       
        verbose("reward states: ",
              "\nposition value: ", self.position_value,
              "\nlong shares: ", self.position_shares,
              "\nshort shares: ", self.shares_owed,
              "\nlong value:", row["Close"] * self.position_shares,
              "\nshort debt:", row["Close"] * self.shares_owed,
              "\ncash value:", self.cash,
              "\ntoal value:", current_portfolio_value,
              "\nbenchmark value:", self.benchmark_value,
              "\nreal percentage_change: ", percentage_change
             )
        
        return percentage_change
        

    def total_value(self):
      self.update_position_value()
      return self.position_value + self.cash
   
    def env_block(self):
      start_index = self.current_index
      end_index = self.current_index
      df = self.timeseries.reset_index().drop(['Date'], axis=1)
      block = df.iloc[start_index].to_numpy()
      return block


In [10]:
def full_train(curriculum_code, num_stocks, create=True):
    product_data = get_data_for_training(num_stocks)
    all_envs = [create_env(curriculum_code) for _ in range(len(product_data))]
    sorted_envs = sorted(all_envs, key=lambda env: env.product)
    if create:
      model = RecurrentPPO(
          "MlpLstmPolicy",
          sorted_envs[0],
          ent_coef=0.1,
          clip_range=0.3,
          verbose=1,
      )
    else:
      model = RecurrentPPO.load(os.path.join(model_path, "baseline-recurrent-ppo"))
    for env in sorted_envs:
      model.set_env(env)
      model.learn(total_timesteps=35000)
      model.save(os.path.join(model_path, "baseline-recurrent-ppo"))


def full_test(symbol, tail=1, head=-1):
    tickerObj = yf.download(tickers = symbol, interval = "1d")
    df = pd.DataFrame(tickerObj)
    if tail !=-1:
      df = df.tail(tail)
    if head !=-1:
      df = df.head(head)

    df = df.reset_index()
    env = TraderEnv(symbol, df)
    model = RecurrentPPO.load(os.path.join(model_path, "baseline-recurrent-ppo"))
    obs, _ = env.reset()
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    done = False
    while not done:
      action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True)
      obs, rewards, _, done, extra = env.step(action)
      episode_starts = done

def make_env_for(symbol, code, tail=-1, head=-1):
  tickerObj = yf.download(tickers = symbol, interval = "1d")
  df = pd.DataFrame(tickerObj)  
  if tail !=-1:
    df = df.tail(tail)
  if head !=-1:
    df = df.head(head)
  df = df.reset_index()  
  env = TraderEnv(symbol, df, code)
  return env


def partial_test(env):
  model = RecurrentPPO.load(os.path.join(model_path, "baseline-recurrent-ppo"))
  obs = env.reset_test()
  done = False
  state = None
  while not done:
    action, state = model.predict(obs, state, episode_start=False, deterministic=False)
    # take the action and observe the next state and reward
    obs, reward, _, done, info_ = env.step(action)
    
def partial_train(env, steps=500, create=False):
  if create:
    model = RecurrentPPO(
        "MlpLstmPolicy",
        env,
        ent_coef=0.1,
        clip_range=0.3,
        verbose=1,
    )
  else:
    model = RecurrentPPO.load(os.path.join(model_path, "baseline-recurrent-ppo"))
  
  model.set_env(env)
  model.learn(total_timesteps=steps)
  model.save(os.path.join(model_path, "baseline-recurrent-ppo"))


In [11]:
import os
import sys


train_model = False
test_model = False
env_count = 0
def create_env(code=1):
    global env_count
    products = list(product_data.keys())
    product = products[env_count]
    env = TraderEnv(product, product_data[product], code)
    env_count+=1
    return env

def timestr():
  import datetime
  current_time = datetime.datetime.now()
  formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S") 
  return formatted_time

if train_model:
  full_train(1, 24)
elif test_model:
  full_test("AAPL")
else:
  print("no action")

def back_test_expert():
  env = make_env_for("TSLA", 1, 365)
  env.expert_opinion_df()
  obs, _ = env.reset()
  
  action = obs[-1]
  done = False
  while not done:
    print("Action:", action)
    state, reward, _, done, _ = env.step(int(action))
    action = state[-1]
    print("Reward:", reward, " for action: ", action, "on probability: ", state[3])
  return env

no action


In [ ]:
from sb3_contrib.ppo_recurrent.policies import MlpLstmPolicy
from typing import NamedTuple, Tuple
import torch as th

class RNNStates(NamedTuple):
    pi: Tuple[th.Tensor, ...]
    vf: Tuple[th.Tensor, ...]

class CustomActorCriticPolicy(MlpLstmPolicy):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.custom_actions = None
        self.custom_actions_index = 0
    def set_custom_actions(self, custom_actions):
        self.custom_actions = custom_actions
        self.custom_actions_index = 0

    def forward(
        self,
        obs,
        lstm_states,
        episode_starts,
        deterministic = False):
        """
        Forward pass in all the networks (actor and critic)

        :param obs: Observation. Observation
        :param lstm_states: The last hidden and memory states for the LSTM.
        :param episode_starts: Whether the observations correspond to new episodes
            or not (we reset the lstm states in that case).
        :param deterministic: Whether to sample or use deterministic actions
        :return: action, value and log probability of the action
        """
        # Preprocess the observation if needed
        features = self.extract_features(obs)
        if self.share_features_extractor:
            pi_features = vf_features = features  # alis
        else:
            pi_features, vf_features = features
        # latent_pi, latent_vf = self.mlp_extractor(features)
        latent_pi, lstm_states_pi = self._process_sequence(pi_features, lstm_states.pi, episode_starts, self.lstm_actor)
        if self.lstm_critic is not None:
            latent_vf, lstm_states_vf = self._process_sequence(vf_features, lstm_states.vf, episode_starts, self.lstm_critic)
        elif self.shared_lstm:
            # Re-use LSTM features but do not backpropagate
            latent_vf = latent_pi.detach()
            lstm_states_vf = (lstm_states_pi[0].detach(), lstm_states_pi[1].detach())
        else:
            # Critic only has a feedforward network
            latent_vf = self.critic(vf_features)
            lstm_states_vf = lstm_states_pi

        latent_pi = self.mlp_extractor.forward_actor(latent_pi)
        latent_vf = self.mlp_extractor.forward_critic(latent_vf)

        # Evaluate the values for the given observations
        values = self.value_net(latent_vf)
        distribution = self._get_action_dist_from_latent(latent_pi)
        if self.custom_actions is not None:
            # Retrieve the next custom action
            custom_action = self.custom_actions[self.custom_actions_index]
            self.custom_actions_index+=1
            if self.custom_actions_index >= len(self.custom_actions):
              self.custom_actions_index=0
            
            actions = np.array([custom_action])
            actions = torch.tensor(actions, dtype=torch.float32, device=self.device)

            # Compute the log probabilities of the custom actions
            log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
        else:
            # If no custom actions are provided, use the actions generated by the actor network
            actions = distribution.get_actions(deterministic=deterministic)
            log_prob = distribution.log_prob(actions)
        return actions, values, log_prob, RNNStates(lstm_states_pi, lstm_states_vf)
        

def guided_training(symbol, create, tail=365, head=-1):
  env = make_env_for(symbol, 1, tail, head)
  if create:
    ppo_agent = RecurrentPPO(
        CustomActorCriticPolicy,
        env,
        ent_coef=0.1,
        clip_range=0.3,
        verbose=1,
    )
  else:
    ppo_agent = RecurrentPPO.load(os.path.join(model_path, "baseline-recurrent-ppo"))

  # generate the actions we want IMPLEMENT ME
  state_action_data = env.expert_opinion() 
  state_action_data = env.expert_opinion()
  custom_actions = [action for _, action in state_action_data]
  ppo_agent.policy.custom_actions = custom_actions
  ppo_agent.learn(total_timesteps=100000)
  ppo_agent.save(os.path.join(model_path, "baseline-recurrent-ppo"))
  return env

env = guided_training("TSLA", True)

[*********************100%***********************]  1 of 1 completed
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
_step: 0  action:  0
holding action = 0.
action: 0  prob: 0.831020603231027
current reward: -1
_step: 1  action:  1
opening long.
action: 1  prob: 0.7564644150039741
current reward: -2
_step: 2  action:  0
holding action = 0.
action: 0  prob: 0.7423195179417588
current reward: -1.5
_step: 3  action:  0
holding action = 0.
action: 0  prob: 0.8412373871160012
current reward: -2.5
_step: 4  action:  1
holding long.
action: 1  prob: 0.8078279530127386
current reward: -1.5
_step: 5  action:  1
holding long.
action: 1  prob: 0.7114149837667222
current reward: -2.5
_step: 6  action:  0
holding action = 0.
action: 0  prob: 0.619257125860244
current reward: -2.0
_step: 7  action:  0
holding action = 0.
action: 0  prob: 0.572799978607782
current reward: -1.5
_step: 8  action:  0
holding action = 0.
action: 0  prob: 0.4456032278537141


<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipytho

current reward: 4.5
_step: 20  action:  0
holding action = 0.
action: 0  prob: 0.4334341164778662
current reward: 5.0
_step: 21  action:  0
holding action = 0.
action: 0  prob: 0.5707153214156949
current reward: 5.5
_step: 22  action:  0
holding action = 0.
action: 0  prob: 0.5596143892661292
current reward: 6.0
_step: 23  action:  0
holding action = 0.
action: 0  prob: 0.6871351183575906
current reward: 6.5
_step: 24  action:  0
holding action = 0.
action: 0  prob: 0.7147340096702808
current reward: 7.0
_step: 25  action:  0
holding action = 0.
action: 0  prob: 0.5334809755349998
current reward: 7.5
_step: 26  action:  0
holding action = 0.
action: 0  prob: 0.6821898461640474
current reward: 8.0
_step: 27  action:  0
holding action = 0.
action: 0  prob: 0.7084136946072825
current reward: 8.5
_step: 28  action:  0
holding action = 0.
action: 0  prob: 0.7837440783279006
current reward: 9.0
_step: 29  action:  0
holding action = 0.
action: 0  prob: 0.7375013969522934
current reward: 9.5


<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipytho

_step: 48  action:  0
holding action = 0.
action: 0  prob: 0.41003877604715666
current reward: 11.5
_step: 49  action:  0
holding action = 0.
action: 0  prob: 0.38767659750076233
current reward: 12.0
_step: 50  action:  0
holding action = 0.
action: 0  prob: 0.5499057817892119
current reward: 12.5
_step: 51  action:  0
holding action = 0.
action: 0  prob: 0.5830475378523643
current reward: 13.0
_step: 52  action:  0
holding action = 0.
action: 0  prob: 0.5691272371038134
current reward: 13.5
_step: 53  action:  0
holding action = 0.
action: 0  prob: 0.5493873682969774
current reward: 14.0
_step: 54  action:  0
holding action = 0.
action: 0  prob: 0.78357225855022
current reward: 14.5
_step: 55  action:  0
holding action = 0.
action: 0  prob: 0.8126146598503298
current reward: 13.5
_step: 56  action:  1
closing short to open long.
action: 1  prob: 0.5879034599982791
current reward: 12.5
_step: 57  action:  0
holding action = 0.
action: 0  prob: 0.5875417236176437
current reward: 13.0
_s

...

<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipytho

_step: 82  action:  0
holding action = 0.
action: 0  prob: 0.7734998801547193
current reward: -3.5
_step: 83  action:  0
holding action = 0.
action: 0  prob: 0.812909740109876
current reward: -4.5
_step: 84  action:  1
holding long.
action: 1  prob: 0.9133481834270067
current reward: -3.5
_step: 85  action:  1
holding long.
action: 1  prob: 0.9411446823401566
current reward: -2.5
_step: 86  action:  1
holding long.
action: 1  prob: 0.9057808598548525
current reward: -1.5
_step: 87  action:  1
holding long.
action: 1  prob: 0.8372068501587253
current reward: -0.5
_step: 88  action:  1
holding long.
action: 1  prob: 0.7999691779563516
current reward: -1.5
_step: 89  action:  0
holding action = 0.
action: 0  prob: 0.7106553365736432
current reward: -1.0
_step: 90  action:  0
holding action = 0.
action: 0  prob: 0.5769527309807619
current reward: -0.5
_step: 91  action:  1
holding long.
action: 1  prob: 0.3549397331516375
current reward: -1.5
_step: 92  action:  1
holding long.
action: 1  

<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipytho

_step: 106  action:  0
holding action = 0.
action: 0  prob: 0.12153030524591868
current reward: -7.5
_step: 107  action:  2
closing long to open short
opening short.
action: 2  prob: 0.0597761595937123
current reward: -6.5
_step: 108  action:  2
holding short.
action: 2  prob: 0.07880105980356333
current reward: -5.5
_step: 109  action:  2
holding short.
action: 2  prob: 0.07721642496832637
current reward: -4.5
_step: 110  action:  2
holding short.
action: 2  prob: 0.04992968204202031
current reward: -3.5
_step: 111  action:  2
holding short.
action: 2  prob: 0.07552949435817635
current reward: -2.5
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 364           |
|    ep_rew_mean          | 6.2e+03       |
| time/                   |               |
|    fps                  | 91            |
|    iterations           | 35            |
|    time_elapsed         | 49            |
|    total_timesteps      | 4480         

<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipython-input-22-c49afe8c100c>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_prob = distribution.log_prob(torch.tensor(actions, dtype=torch.float32, device=self.device))
<ipytho

In [ ]:
print(env.cash)
env.ledger